# Deutsch-Jozsa Algorithm

In this tutorial we introduce one of the first quantum algorithm’s developed by pioneers David Deutsch and Richard Jozsa. This algorithm showcases an efficient quantum solution to a problem that cannot be solved classically but instead can be solved using a quantum device.   

## References 
[1] [Michael A. Nielsen & Isaac L. Chuang Quantum Computation and Quantim Information textbook](http://mmrc.amss.cas.cn/tlb/201702/W020170224608149940643.pdf)


## Deutsch's Problem 
The algorithm proposed is solving a black box problem, trying to determine what the black box is the task at hand. The problem itself is not very applicable in real life but showcases an occasion where a quantum algorithm performs exponentially better than a classical one. 

Let $U_f$ be an oracle (black box) that computes a Boolean function which only takes binary inputs (0’s or 1’s). These functions can be represented as $f: {0, 1}^n →  {0, 1}$. This oracle evaluates two types of functions, constant or balanced. 

A constant function takes any input and returns only 0’s or only 1’s and a balanced function takes any input and returns exactly half 0’s and half 1’s. 

**Constant:** 011010 → $U_f$ → 000000  
**Balanced:** 000011 → $U_f$ → 111000

The goal is to determine what type of function is $U_f$ based on only the outputs. If the input is as large as $2^n$ then the amount of queries a classical computer will have to make is $2n/2+1$. We can see for a large enough n this problem scales exponentially and becomes inefficient to solve classically. However, leveraging a quantum algorithm we only need to query the oracle once to determine the type of function for $U_f$. This is possible because the state of its output might be in a coherent superposition of states corresponding to different answers, each which solves the problem (Deutsch 1992). 


In [12]:
import matplotlib.pyplot as plt

%matplotlib inline

import random

import numpy as np

from braket.experimental.algorithms.deutsch_jozsa.deutsch_jozsa import (
    balanced_oracle,
    classical_generator,
    constant_oracle,
    deutsch_jozsa_algorithm,
    init_states,
    quantum_generator,
    random_oracle,
)


The initalization function prepares a superposition of all possible input values and the second register is in a superposition of 0 and 1. 

In [13]:
n_qubits = 3

In [14]:
# example circuit initializition for 3 qubits
init = init_states(n_qubits)
print(init)

T  : |0|1|
          
q0 : -H---
          
q1 : -H---
          
q2 : -H---
          
q3 : -X-H-

T  : |0|1|


The oracle is the Boolean function that is applied to the $n$-qubits in the query register. 

### For a constant function 
1. if $f(x) = 0$, then apply the I gate to the qubit in second register.  
2. if $f(x) = 1$, then apply the X gate to the qubit in second register.  

In [15]:
# example circuit for constant oracle
constant = constant_oracle(n_qubits)
print(constant)

T  : |0|
        
q3 : -I-

T  : |0|


---------------------------------------------------------------------------------------------------------------------
### For a balanced function

Apply a CNOT gate to every qubit in the first register and set the qubit in the second register as the target. Use a random number generator to randomly choose which qubits to add an X - gate before and after the CNOT gate.

In [16]:
# example circuit for balanced oracle
balanced = balanced_oracle(n_qubits)
print(balanced)

T  : |0|1|2|3|
              
q0 : -X-C-X---
        |     
q1 : -X-|-C-X-
        | |   
q2 : ---|-|-C-
        | | | 
q3 : ---X-X-X-

T  : |0|1|2|3|


### For a random function

We have created a random oracle generator to showcase the many different ways a constant or balanced oracle can be made using just X, CNOT and CCNOT gates. When running this function, you can have 3 different outcomes. Either a constant function, a balanced function or neither. 

In [17]:
random = random_oracle(n_qubits)
print(random)

T  : |0|1|2|
            
q1 : -X-C-X-
        |   
q2 : ---X---

T  : |0|1|2|


### General Deutsch-Jozsa algorithm

In [18]:
# randomly choose the type of oracle to implement in your algorithm
dj_circuit, dj_result = deutsch_jozsa_algorithm(
    np.random.choice([constant, balanced, random]), n_qubits
)
print(dj_circuit)

Measurement results:
 {'0001': 0.5008, '0000': 0.4992}
T  : |0|1|2|3|4|Result Types|
                           
q0 : -H-X-C-X-H-Sample(Z)----
          |                
q1 : -H---X-H---Sample(Z)----
                           
q2 : -H-H-------Sample(Z)----
                           
q3 : -X-H--------------------

T  : |0|1|2|3|4|Result Types|


In [19]:
print(dj_result.measurement_counts)

Counter({'0001': 5008, '0000': 4992})


## Classicaly Run the Random Circuit Generator 

The function classical_generator takes a random oracle and applies it to $2^n$ possible combinations where n is the number of qubits you've initialized. We can create a classical circuit by removing the Hadamard gates because without putting any qubits in superposition, the circuit can't display quantum mechanical properties. 

If n = 3 then we get a result of 8 different circuits. We then determine if these circuits are constant, balanced or neither at the end so we can compare it with results when we run this circuit but this time on a quantum circuit.

#### *To generate a new random circuit run the [random oracle](#random) again!*

In [20]:
run_circuit = classical_generator(n_qubits, random)

[0, 0, 0]
T  : |0|1|2|3|Result Types|
                         
q0 : -I-X-C-X--------------
          |              
q1 : -I---X----------------
                         
q2 : -I--------------------
                         
q3 : ---------Sample(Z)----

T  : |0|1|2|3|Result Types|
Measurement results:
 {'0100': 1.0}
[0, 0, 1]
T  : |0|1|2|3|Result Types|
                         
q0 : -I-X-C-X--------------
          |              
q1 : -I---X----------------
                         
q2 : -X--------------------
                         
q3 : ---------Sample(Z)----

T  : |0|1|2|3|Result Types|
Measurement results:
 {'0110': 1.0}
[0, 1, 0]
T  : |0|1|2|3|Result Types|
                         
q0 : -I-X-C-X--------------
          |              
q1 : -X---X----------------
                         
q2 : -I--------------------
                         
q3 : ---------Sample(Z)----

T  : |0|1|2|3|Result Types|
Measurement results:
 {'0000': 1.0}
[0, 1, 1]
T  : |0|1|2|3|Result Types|
     

## Run Random Circuit Generator on Quantum Circuit

Now we can run the random circuit we generated classically on a quantum circuit to compare and see if the results are the same. We measure the results and based on the output we can determine what type of function the random circuit was.

In [21]:
oracle = quantum_generator(n_qubits, run_circuit)

Measurement results:
 {'0001': 0.46, '0000': 0.54}
This is a constant function


<a id="summary"></a>

## Comparing the Classical and Quantum Results

Once you've run the classical and quantum version of the random circuit you can compare the results. If both correspond to the same type of function then we have found another way to implement the constant or balanced function! 

You can keep generating more random circuits to see how the constant and balanced functions can be created including circuits that are extremely large, for example 10 qubits! There is also a possibility that you can generate a random circuit that is neither a constant or balanced function. 

To test new random circuits, remember to run the [random oracle](#random) once again.

In [22]:
from braket.tracking import Tracker

tracker = Tracker()

print("Task Summary")
print(tracker.quantum_tasks_statistics())
print(
    "Note: Charges shown are estimates based on your Amazon Braket simulator and quantum processing unit (QPU) task usage. Estimated charges shown may differ from your actual charges. Estimated charges do not factor in any discounts or credits, and you may experience additional charges based on your use of other services such as Amazon Elastic Compute Cloud (Amazon EC2)."
)
print(
    f"Estimated cost to run this example: {tracker.qpu_tasks_cost() + tracker.simulator_tasks_cost():.2f} USD"
)

Task Summary
{}
Note: Charges shown are estimates based on your Amazon Braket simulator and quantum processing unit (QPU) task usage. Estimated charges shown may differ from your actual charges. Estimated charges do not factor in any discounts or credits, and you may experience additional charges based on your use of other services such as Amazon Elastic Compute Cloud (Amazon EC2).
Estimated cost to run this example: 0.00 USD
